## Linear Elasticity for Cantilever Beam

In [51]:
using Gridap, Gridap.CellData, SparseArrays, BenchmarkTools
using NearestNeighbors, Printf  # make sure this is at the top of your file
using Makie,GLMakie, LaTeXStrings

In [61]:
using Gridap.Fields
using Gridap.Polynomials
using Gridap.ReferenceFEs
using LinearAlgebra
using FillArrays
using Gridap.Arrays
# nodes = Point{2,Float64}[(0,0),(0,1),(0,2),(1,0),(1,1),(1,2),(2,0),(2,1),(2,2)] 
nodes = Fields.Point{2,Float64}[(0, 0), (1, 0), (2, 0), (0, 1), (1, 1), (2, 1), (0, 2), (1, 2), (2, 2)] # Avoi Point width point in Makie.Point
conn = [[1, 2, 4, 5], [2, 3, 5, 6], [4, 5, 7, 8], [5, 6, 8, 9]]

# Reference shape functions 
filter(e, p) = true
ref_nodes = Point{2,Float64}[
    (0, 0), (1, 0), (0, 1), (1, 1)]

m = MonomialBasis{2}(Float64, 1, filter)
l = LagrangianDofBasis(Float64, ref_nodes)
change = inv(evaluate(l, m))
s = linear_combination(change, m)
q = Point{2,Float64}[(0.5, 0.5)]
w = [1.]

# Cell-wise shape functions 
ncells = length(conn)
cell_s = Fill(s, ncells)
# Cell-wise quadrature 
cell_q = Fill(q, ncells)
cell_w = Fill(w, ncells)
# Geometrical map and Jacobian 
cell_nodes = lazy_map(Broadcasting(Arrays.Reindex(nodes)), conn)

cell_φ = lazy_map(
    linear_combination, cell_nodes, cell_s)
cell_Jt = lazy_map(∇, cell_φ)
# Cell-wise shape function derivatives
cell_∇ref_s = lazy_map(Broadcasting(∇), cell_s)
cell_invJt = lazy_map(Operation(inv), cell_Jt)
cell_∇s = lazy_map( Broadcasting(Operation(⋅)), cell_invJt, cell_∇ref_s)
# Cell-wise stifness matrix 50 cell_∇st = lazy_map(transpose,cell_∇s) 51 cell_∇s∇st = lazy_map( 52
 cell_∇st = lazy_map(transpose, cell_∇s)
# cell_∇s∇st = lazy_map(  Broadcasting(Operation(⋅)), cell_∇s, cell_∇st)
# cell_mat = lazy_map( integrate, cell_∇s∇st, cell_q, cell_w, cell_Jt)
# cell_mat[1]


4-element LazyArray{Fill{typeof(transpose), 1, Tuple{Base.OneTo{Int64}}}, Transpose{Gridap.Fields.OperationField{typeof(dot), Tuple{Gridap.Fields.OperationField{typeof(inv), Tuple{Gridap.Fields.LinearCombinationField{Vector{VectorValue{2, Float64}}, Gridap.Fields.LinearCombinationFieldVector{Matrix{Float64}, FieldGradientArray{1, MonomialBasis{2, Float64}, FieldGradient{1, Gridap.Polynomials.Monomial}, 1}}}}}, Gridap.Fields.LinearCombinationField{Matrix{Float64}, FieldGradientArray{1, MonomialBasis{2, Float64}, FieldGradient{1, Gridap.Polynomials.Monomial}, 1}}}}, Gridap.Fields.BroadcastOpFieldArray{typeof(dot), Gridap.Fields.OperationField{typeof(dot), Tuple{Gridap.Fields.OperationField{typeof(inv), Tuple{Gridap.Fields.LinearCombinationField{Vector{VectorValue{2, Float64}}, Gridap.Fields.LinearCombinationFieldVector{Matrix{Float64}, FieldGradientArray{1, MonomialBasis{2, Float64}, FieldGradient{1, Gridap.Polynomials.Monomial}, 1}}}}}, Gridap.Fields.LinearCombinationField{Matrix{Float6

In [62]:
cell_∇st

4-element LazyArray{Fill{typeof(transpose), 1, Tuple{Base.OneTo{Int64}}}, Transpose{Gridap.Fields.OperationField{typeof(dot), Tuple{Gridap.Fields.OperationField{typeof(inv), Tuple{Gridap.Fields.LinearCombinationField{Vector{VectorValue{2, Float64}}, Gridap.Fields.LinearCombinationFieldVector{Matrix{Float64}, FieldGradientArray{1, MonomialBasis{2, Float64}, FieldGradient{1, Gridap.Polynomials.Monomial}, 1}}}}}, Gridap.Fields.LinearCombinationField{Matrix{Float64}, FieldGradientArray{1, MonomialBasis{2, Float64}, FieldGradient{1, Gridap.Polynomials.Monomial}, 1}}}}, Gridap.Fields.BroadcastOpFieldArray{typeof(dot), Gridap.Fields.OperationField{typeof(dot), Tuple{Gridap.Fields.OperationField{typeof(inv), Tuple{Gridap.Fields.LinearCombinationField{Vector{VectorValue{2, Float64}}, Gridap.Fields.LinearCombinationFieldVector{Matrix{Float64}, FieldGradientArray{1, MonomialBasis{2, Float64}, FieldGradient{1, Gridap.Polynomials.Monomial}, 1}}}}}, Gridap.Fields.LinearCombinationField{Matrix{Float6

In [56]:
# Constitutive law for LINEAR ELASTICITY
const ν = 0.3   # Poisson's ratio
const E = 119e3 # Young's modulus
const λ = (E * ν) / ((1 + ν) * (1 - 2 * ν)) # Lame's first parameter
const μ = E / (2 * (1 + ν)) # Shear modulus
ε₀(u) = 0.5 * (∇(u) + transpose(∇(u))) # Small strain tensor
σ(ε) = λ * tr(ε) * one(ε) + 2μ * ε # Stress tensor

# Cell-wise strain and stress
cell_ε₀ = lazy_map(ε₀, cell_∇s)
# cell_σ = lazy_map(σ, cell_ε₀)

# # Cell-wise bilinear form contribution
# cell_ε₀v = lazy_map(ε₀, cell_∇s) # Strain of test function v
# cell_integrand = lazy_map(Broadcasting(Operation(⊙)), cell_σ, cell_ε₀v)
# cell_mat = lazy_map(integrate, cell_integrand, cell_q, cell_w, cell_Jt)

ErrorException: 

Function gradient (aka ∇) is not defined for arrays of Field objects.
Use Broadcasting(∇) instead.


OperationField()

4-element LazyArray{Fill{Broadcasting{Operation{typeof(dot)}}, 1, Tuple{Base.OneTo{Int64}}}, Gridap.Fields.BroadcastOpFieldArray{typeof(dot), Gridap.Fields.OperationField{typeof(dot), Tuple{Gridap.Fields.OperationField{typeof(inv), Tuple{Gridap.Fields.LinearCombinationField{Vector{Gridap.TensorValues.VectorValue{2, Float64}}, Gridap.Fields.LinearCombinationFieldVector{Matrix{Float64}, FieldGradientArray{1, MonomialBasis{2, Float64}, FieldGradient{1, Gridap.Polynomials.Monomial}, 1}}}}}, Gridap.Fields.LinearCombinationField{Matrix{Float64}, FieldGradientArray{1, MonomialBasis{2, Float64}, FieldGradient{1, Gridap.Polynomials.Monomial}, 1}}}}, 1, Tuple{Gridap.Fields.OperationField{typeof(inv), Tuple{Gridap.Fields.LinearCombinationField{Vector{Gridap.TensorValues.VectorValue{2, Float64}}, Gridap.Fields.LinearCombinationFieldVector{Matrix{Float64}, FieldGradientArray{1, MonomialBasis{2, Float64}, FieldGradient{1, Gridap.Polynomials.Monomial}, 1}}}}}, Gridap.Fields.LinearCombinationFieldVect

4×4 Matrix{Float64}:
  0.5   0.0   0.0  -0.5
  0.0   0.5  -0.5   0.0
  0.0  -0.5   0.5   0.0
 -0.5   0.0   0.0   0.5

## Inputs

In [ ]:
L = 20 # Length
W = 10 # Width and Height
domain = (0, L, 0, W, 0, W) # sizes along X, Y, Z
partition = (20, 10, 10) # mesh sizes

### Add labels to left and right faces for boundary conditions

In [ ]:
model = CartesianDiscreteModel(domain, partition);
# Tag the left face (x = 0); right face (x = L)
labels = get_face_labeling(model)
add_tag_from_tags!(labels,"left", [1,3,5,7,13,15,17,19,25]); # left face: corners (1,3,5,7); edges (13,15,17,19); others (25) 
add_tag_from_tags!(labels, "right", [2,4,6,8,14,16,18,20,26]); # Right face: corners (2,4,6,8); edges (14,16,18,20); others (26) 
# writevtk(Ω,"Cantilever_Beam_3D")

### Construct weak form in Gridap.jl

In [ ]:
# Integrating on the domain Ω
degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω, degree)
# neumann boundaries
Γ = BoundaryTriangulation(model,tags = "right")
dΓ = Measure(Γ,degree)
# Vector-valued FE space
order = 1
reffe = ReferenceFE(lagrangian, VectorValue{3,Float64}, order)

V = TestFESpace(Ω, reffe; conformity=:H1, dirichlet_tags = "left")
U = TrialFESpace(V)

# Constitutive law for LINEAR ELASTICITY 
const ν = 0.3   # Poisson's ratio
const E = 119e3 # Young's modulus

const λ = (E*ν)/((1+ν)*(1-2*ν)) # C12
const μ = E/(2*(1+ν)) # Shear modulus
ε₀(u) = 0.5 * ( ∇(u) + transpose(∇(u))) # Strain; In Gridap this can be automatically defined in Gridap.ε
σ(ε₀) = λ * tr(ε₀) * one(ε₀) + 2μ * ε₀; # Stress
# The weak form
F(x) = VectorValue(0.0, -1, 0.0) # A unit load along Y axis applies to each node on the right face
a(u,v) = ∫((σ∘ε₀(u)) ⊙ ε₀(v)) *dΩ # Left-hand size; (∘) Composite functions
l(v) = ∫(F ⋅ v) * dΓ # Right-hand size

## Run finite element analysis

In [ ]:

# Solution of the FE problem
op = AffineFEOperator(a,l,U,V)
uh = solve(op)
# writevtk(Ω,"Cantilever_Beam_3D_uh",cellfields=["uh"=>uh,"epsi"=>ε₀(uh),"sigma"=>σ∘ε₀(uh)])

### Compute the compliance

In [ ]:
# Method 1
C1 = sum(∫((σ ∘ ε₀(uh)) ⊙ ε₀(uh)) * dΩ) # Compliance
println("Compliance 1: ", C1)
# Method 2
C2 = sum(∫(F ⋅ uh) * dΓ) # Compliance
println("Compliance 2: ", C2)
# Method 3
U = get_free_dof_values(uh) # displacement vector
K = get_matrix(op) # stiffness matrix
C3 = dot(U, K * U)
println("Compliance 3: ", C3)
# Method 4 using for loop to compute sum(ue^T*ke*ue)

using Gridap.Arrays, Gridap.FESpaces

U_e = get_cell_dof_values(uh)
u = get_trial_fe_basis(V)
v = get_fe_basis(V)
K_e = get_array(∫((σ ∘ ε₀(u)) ⊙ ε₀(v)) * dΩ)
# ==
a1 = ∫((σ ∘ ε₀(u)) ⊙ ε₀(v)) * dΩ
KK = a1[Ω]
KK[1] .- K_e[1]
assem = SparseMatrixAssembler(V,V)
# Allocation global parametric stiffness
cell_dofs = get_cell_dof_ids(V)
data = ([KK],[cell_dofs],[cell_dofs])
KK_Global = allocate_matrix(assem,data)
# =========
numele = num_cells(Ω) # Number of elements
C4 = 0.0 # Comliance
for e=1:numele
    ue = U_e[e]    # element displacement
    # ke = KK[e] # element stiffness matrix
    ke = K_e[e] # element stiffness matrix
    C4 = C4 + transpose(ue)*ke*ue
end
println("Compliance 4: ", C4)


In [ ]:
# Function to compute the B matrix at a quadrature point
function compute_B(ε_u)
    num_dofs = length(ε_u)  # Number of degrees of freedom (DOFs) in the element
    B = zeros(6, num_dofs)  # Initialize B matrix (6 strain components for 3D)
    for j in 1:num_dofs
        ε_j = ε_u[j]  # Symmetric tensor for the j-th basis function
        B[1, j] = ε_j[1,1]  # ε_xx
        B[2, j] = ε_j[2,2]  # ε_yy
        B[3, j] = ε_j[3,3]  # ε_zz
        B[4, j] = 2 * ε_j[1,2]  # 2ε_xy (shear strain γ_xy)
        B[5, j] = 2 * ε_j[2,3]  # 2ε_yz (shear strain γ_yz)
        B[6, j] = 2 * ε_j[3,1]  # 2ε_zx (shear strain γ_zx)
    end
    return B
end

# Compute B matrix for each cell
u = get_trial_fe_basis(V)

cell_values = get_array(∫(ε₀(u)) * dΩ)

ε_u=cell_values[1]

num_nodes = 8       # Linear tetrahedral elements have 4 nodes
B1 = compute_B(ε_u)

cell_values = get_array(∫(ε₀(uh)) * dΩ)
ε_u=cell_values[1]

ε_u


# # # Example: Access the B matrix for the first cell
# B_first_cell = cell_B_matrices[1]
# println("Strain-displacement matrix for the first cell:\n", B_first_cell)


In [ ]:
B1

In [ ]:
using Gridap

# Assuming V is the finite element space and dΩ is the integration measure
u = get_trial_fe_basis(V)  # Get the trial basis functions from the FE space
q = get_cell_points(dΩ)    # Get the quadrature points for each cell

# Evaluate the symmetric gradient of the basis functions at the quadrature points
ε_u_at_q = evaluate(ε₀(u), q)

# Function to compute the B matrix from symmetric gradients at a quadrature point
function compute_B(ε_u)
    num_dofs = length(ε_u)  # Number of degrees of freedom (DOFs) in the element
    B = zeros(6, num_dofs)  # Initialize B matrix (6 strain components for 3D)
    for j in 1:num_dofs
        ε_j = ε_u[j]  # Symmetric tensor for the j-th basis function
        B[1, j] = ε_j[1,1]  # ε_xx
        B[2, j] = ε_j[2,2]  # ε_yy
        B[3, j] = ε_j[3,3]  # ε_zz
        B[4, j] = 2 * ε_j[1,2]  # 2ε_xy (shear strain γ_xy)
        B[5, j] = 2 * ε_j[2,3]  # 2ε_yz (shear strain γ_yz)
        B[6, j] = 2 * ε_j[3,1]  # 2ε_zx (shear strain γ_zx)
    end
    return B
end

# Compute the B matrix for each cell at each quadrature point
cell_B = lazy_map(ε_u_q -> [compute_B(ε_u_q[qp, :]) for qp in 1:size(ε_u_q, 1)], ε_u_at_q)

# Example: Access and display the B matrix for the first cell at the first quadrature point
B_first_cell_first_qp = cell_B[1][1]
println("B matrix for the first cell at the first quadrature point:\n", B_first_cell_first_qp)

In [ ]:
using Gridap

# Define the 3D domain and mesh (example: Cartesian grid)
domain = (0.0, 1.0, 0.0, 1.0, 0.0, 1.0)
partition = (2, 2, 2)  # 2x2x2 elements (adjust as needed)
model = CartesianDiscreteModel(domain, partition)

# Triangulation and integration measure
Ω = Triangulation(model)
dΩ = Measure(Ω, 2)  # Quadrature degree 2 for linear hex elements

# Define the finite element space for vector-valued displacements
order = 1
reffe = ReferenceFE(HEX, lagrangian, VectorValue{3,Float64}, order)
V = TestFESpace(model, reffe; conformity=:H1)

In [ ]:
# Material properties
E = 119e3  # Young's modulus (Pa)
ν = 0.3    # Poisson's ratio
λ = (E * ν) / ((1 + ν) * (1 - 2ν))  # Lamé's first parameter
μ = E / (2 * (1 + ν))               # Shear modulus

# Constitutive matrix D for 3D (Voigt notation)
D = [
    λ + 2μ  λ     λ     0.0   0.0   0.0;
    λ       λ + 2μ λ     0.0   0.0   0.0;
    λ       λ     λ + 2μ 0.0   0.0   0.0;
    0.0     0.0   0.0   μ     0.0   0.0;
    0.0     0.0   0.0   0.0   μ     0.0;
    0.0     0.0   0.0   0.0   0.0   μ
]

In [ ]:
# Get trial basis functions and quadrature points
u = get_trial_fe_basis(V)
q = get_cell_points(dΩ)
ε_u_at_q = evaluate(ε₀(u), q)  # Shape: (num_quad_points, num_cells, num_dofs)

# Function to compute B_e for a given quadrature point
function compute_Be(ε_u)
    num_nodes = 8
    num_dofs = num_nodes * 3  # 24 DOFs
    B = zeros(6, num_dofs)
    for k in 1:num_nodes
        for α in 1:3  # x, y, z components
            j = (k-1)*3 + α
            ε_j = ε_u[j]  # Access the tensor for the j-th DOF
            if α == 1  # u_x at node k
                B[1,j] = ε_j[1,1]  # ∂N_k/∂x for ε_xx
                B[4,j] = 2 * ε_j[1,2]  # 2 * ∂N_k/∂y for γ_xy
                B[6,j] = 2 * ε_j[1,3]  # 2 * ∂N_k/∂z for γ_zx
            elseif α == 2  # u_y at node k
                B[2,j] = ε_j[2,2]  # ∂N_k/∂y for ε_yy
                B[4,j] = 2 * ε_j[2,1]  # 2 * ∂N_k/∂x for γ_xy
                B[5,j] = 2 * ε_j[2,3]  # 2 * ∂N_k/∂z for γ_yz
            elseif α == 3  # u_z at node k
                B[3,j] = ε_j[3,3]  # ∂N_k/∂z for ε_zz
                B[5,j] = 2 * ε_j[3,2]  # 2 * ∂N_k/∂y for γ_yz
                B[6,j] = 2 * ε_j[3,1]  # 2 * ∂N_k/∂x for γ_zx
            end
        end
    end
    return B
end

# Compute B_e for each quadrature point in each cell
cell_Be = lazy_map(ε_u_q -> begin
    num_quad_points = size(ε_u_q, 1)  # Number of quadrature points
    [compute_Be(view(ε_u_q, qp, :, :)) for qp in 1:num_quad_points]
end, ε_u_at_q)

# Function to compute B_e^T D B_e
function compute_BTDBe(B, D)
    return B' * D * B
end
cell_Be
# # Compute the integrand B_e^T D B_e
cell_BTDBe = lazy_map(Be_q -> [compute_BTDBe(Be, D) for Be in Be_q], cell_Be)

# # Integrate to get the element stiffness matrix
K_e = integrate(cell_BTDBe, dΩ)

# # Display the result for the first element
# println("Element stiffness matrix K_e for the first element:\n", K_e[1])